In [ ]:
import tiffile
from pathlib import Path
import glob
import numpy as np
from cellmapping.stitching import vertical_stack_images
from matplotlib import pyplot as plt

In [ ]:
IMAGE_DIR = Path("/path_to_brain_images/")
TOP_Y_END = 2157
BOTTOM_Y_START = 819

In [ ]:
rc1_file = glob.glob(str(IMAGE_DIR/"*AP1*.tif"))[0]
rc2_file = glob.glob(str(IMAGE_DIR/"*AP2*.tif"))[0]

In [ ]:
stitched_folder = rc1_file.split('_AP')[0] + '_stitched'

In [ ]:
stitched_folder

In [ ]:
import os
if not os.path.exists(stitched_folder):
    os.makedirs(stitched_folder)

In [ ]:
rc1 = tiffile.imread(rc1_file)

In [ ]:
rc1.shape

In [ ]:
plt.figure(figsize=(24,12))

In [ ]:
plt.imshow(rc1[890,:,:],cmap='gray')

In [ ]:
rc2 = tiffile.imread(rc2_file)

In [ ]:
plt.imshow(rc2[890,:,:],cmap='gray')

In [ ]:
y_offset=600
for z in range(rc1.shape[0]):
    image1 = rc1[z,:,:]
    image2 = rc2[z,:,:]
    stacked_image = np.vstack((image1[:-y_offset,:],image2[y_offset:,:]))
    stacked_image = stacked_image.astype(rc2.dtype)
    output_file = os.path.join(stitched_folder, f"{z:05}.tif")
    tiffile.imwrite(output_file,stacked_image)
    

In [ ]:
os.listdir(stitched_folder)

In [ ]:
from scipy.signal import correlate2d

In [ ]:
image1 = rc1[690,:,:]
image2 = rc2[690,:,:]

In [ ]:
y_offset=600
stacked_image = np.vstack((image1[:-y_offset,:],image2[y_offset:,:]))

In [ ]:
stacked_image = np.vstack((image1[:-y_offset,:],image2[y_offset:,:]))

In [ ]:
plt.figure(figsize=(24,12))
plt.imshow(stacked_image,cmap='gray')

In [ ]:
def mse_quality(img1_values,img2_values):
    return np.mean(img1_values - img2_values) ** 2

In [ ]:
img1_values = image1[-10+(-y_offset):-y_offset,:]
img2_values = image2[y_offset:y_offset+10,:]

In [ ]:
print(mse_quality(img1_values,img2_values))

In [ ]:
min_y = 0
max_y = image1.shape[0]
optimal_y = -1

In [ ]:
min_quality = float('inf')

In [ ]:
for y_offset in range(400,1000):
    img1_values = image1[-y_offset+1,:]
    img2_values = image2[y_offset,:]
    quality = np.sum(img1_values*img2_values)
    if quality <= min_quality:
        optimal_y = y_offset
        min_quality = quality


In [ ]:
print(optimal_y, min_quality)

In [ ]:
image1[-y_offset+1,:],image2[y_offset,:]

In [ ]:
correlation = correlate2d(img1_values,img2_values,mode='same')

In [ ]:
correlation

In [ ]:
y_offset = np.argmax(correlation) - correlation.shape[1] // 2

In [ ]:
y_offset

In [ ]:
if y_offset >= 0:
    stacked_image = np.vstack((image1[:-y_offset,:],image2[y_offset:,:]))
else:
    stacked_image = np.vstack((image1[:y_offset,:],image2[-y_offset:,:]))

In [ ]:
img_combined_890 = vertical_stack_images(rc1[890,:,:],rc2[890,:,:],TOP_Y_END,BOTTOM_Y_START)

In [ ]:
img_combined = vertical_stack_images(rc1,rc2,TOP_Y_END,BOTTOM_Y_START)

In [ ]:
img_combined = img_combined.astype(rc2.dtype)
output_file = rc2_file.replace('_RC2','')
tiffile.imwrite(output_file,img_combined)

In [ ]:
tiffile.imwrite(output_file,img_combined)